In [1]:
import torch

In [2]:
with open('../data/shakespeare.txt', 'r') as f:
    text = f.read()

In [4]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you
